In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from urllib.parse import urlparse
import mlflow
from mlflow.models.signature import infer_signature
import mlflow.sklearn
import dagshub
import logging
import dagshub
import optuna as optuna

dagshub.init(repo_owner='marufmullah50', repo_name='Pytorch---DL_implementation', mlflow=True)

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

c:\Users\Lenovo\anaconda3\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Accessing as marufmullah50

Initialized MLflow to track repo "marufmullah50/Pytorch---DL_implementation"

Repository marufmullah50/Pytorch---DL_implementation initialized!

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import mlflow
import mlflow.pytorch
import optuna

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = pd.read_csv('G:\\Github_Projects\\Pytorch_learning\\mlflow\\data.csv')
x = df.iloc[:, :3].values  # First 3 columns as input features
y = df.iloc[:, 3].values.reshape(-1, 1)  # 4th column as target

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test  = torch.tensor(X_test,  dtype=torch.float32).to(device)

y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test  = torch.tensor(y_test,  dtype=torch.float32).to(device)

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env automatically

True

In [ ]:
def eval_metrics(actual, pred):
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [6]:
def objective(trial):
    # -------------------------------
    # Hyperparameters to tune
    # -------------------------------
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step=8)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    epochs = trial.suggest_int('epochs', 10, 50, step=10)

    # -------------------------------
    # Start MLflow nested run
    # -------------------------------
    with mlflow.start_run(nested=True):
        # Build model with variable hidden layers
        layers = []
        input_dim = X_train.shape[1]
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(input_dim, neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = neurons_per_layer
        layers.append(nn.Linear(input_dim, 1))  # Output layer
        model = nn.Sequential(*layers).to(device)

        # Choose optimizer
        if optimizer_name == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        elif optimizer_name == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        else:  # RMSprop
            optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        loss_fn = nn.MSELoss()

        # Create DataLoader for batch training
        train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # -------------------------------
        # Training loop
        # -------------------------------
        for epoch in range(epochs):
            model.train()
            for xb, yb in train_loader:
                optimizer.zero_grad()
                loss = loss_fn(model(xb), yb)
                loss.backward()
                optimizer.step()

        # -------------------------------
        # Validation loss
        # -------------------------------
        model.eval()
        with torch.no_grad():
            val_loss = loss_fn(model(X_test), y_test).item()

        # -------------------------------
        # Log parameters and metrics
        # -------------------------------
        mlflow.log_params({
            "num_hidden_layers": num_hidden_layers,
            "neurons_per_layer": neurons_per_layer,
            "dropout_rate": dropout_rate,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "optimizer": optimizer_name,
            "weight_decay": weight_decay,
            "epochs": epochs
        })
        mlflow.log_metric("val_loss", val_loss)

        # Log model
        mlflow.pytorch.log_model(model, "model")

    return val_loss


In [8]:
mlflow.set_experiment("Optuna(8parameters)-PyTorch")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

print("Best params:", study.best_params)
print("Best trial:")
print(study.best_trial.params)


2026/02/11 02:33:30 INFO mlflow.tracking.fluent: Experiment with name 'Optuna(8parameters)-PyTorch' does not exist. Creating a new experiment.
[I 2026-02-11 02:33:30,923] A new study created in memory with name: no-name-cab88970-e3c5-4254-9ab3-d61ec7e27076
2026/02/11 02:33:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 02:33:58 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run luxuriant-ox-418 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1/runs/b8fdda08204547b39c4beeb4ea94d3c3
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1


[I 2026-02-11 02:34:03,580] Trial 0 finished with value: 0.146372988820076 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 96, 'dropout_rate': 0.1, 'learning_rate': 0.001178057090322351, 'batch_size': 16, 'optimizer': 'RMSprop', 'weight_decay': 3.8140617177244557e-05, 'epochs': 30}. Best is trial 0 with value: 0.146372988820076.
2026/02/11 02:34:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 02:34:20 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run zealous-shark-8 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1/runs/4e86272faaf742deaf23bbc5c8911820
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1


[I 2026-02-11 02:34:24,916] Trial 1 finished with value: 1.2918353080749512 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 8, 'dropout_rate': 0.4, 'learning_rate': 0.0005034966818896504, 'batch_size': 64, 'optimizer': 'SGD', 'weight_decay': 1.0380277920057428e-05, 'epochs': 20}. Best is trial 0 with value: 0.146372988820076.
2026/02/11 02:34:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 02:34:41 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


...

🏃 View run upset-croc-674 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1/runs/58f6cc0a3b0e41969eca66305a60dcd6
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1


[I 2026-02-11 02:35:24,365] Trial 3 finished with value: 0.18228375911712646 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 88, 'dropout_rate': 0.5, 'learning_rate': 0.015602468311799853, 'batch_size': 64, 'optimizer': 'Adam', 'weight_decay': 0.00017433858321067215, 'epochs': 50}. Best is trial 2 with value: 0.13598349690437317.
2026/02/11 02:35:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 02:35:51 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run adorable-seal-168 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1/runs/395dfe1ded49451eb380079877b0fc5e
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/1


[I 2026-02-11 02:35:55,947] Trial 4 finished with value: 0.4947856664657593 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 64, 'dropout_rate': 0.4, 'learning_rate': 0.04190937228480638, 'batch_size': 64, 'optimizer': 'Adam', 'weight_decay': 0.00016525824397187465, 'epochs': 30}. Best is trial 2 with value: 0.13598349690437317.


Best params: {'num_hidden_layers': 1, 'neurons_per_layer': 64, 'dropout_rate': 0.4, 'learning_rate': 0.006580063844377543, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 1.1197158352562648e-05, 'epochs': 50}
Best trial:
{'num_hidden_layers': 1, 'neurons_per_layer': 64, 'dropout_rate': 0.4, 'learning_rate': 0.006580063844377543, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 1.1197158352562648e-05, 'epochs': 50}


## For Optimizing the r2, rmse

In [7]:
def eval_metrics(actual, pred):
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [12]:
def objective(trial):
    # -------------------------------
    # Hyperparameters to tune
    # -------------------------------
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step=8)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    epochs = trial.suggest_int('epochs', 10, 50, step=10)

    # -------------------------------
    # Start MLflow nested run
    # -------------------------------
    with mlflow.start_run(nested=True):
        # Build model with variable hidden layers
        layers = []
        input_dim = X_train.shape[1]
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(input_dim, neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = neurons_per_layer
        layers.append(nn.Linear(input_dim, 1))  # Output layer
        model = nn.Sequential(*layers).to(device)

        # Choose optimizer
        if optimizer_name == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        elif optimizer_name == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        else:  # RMSprop
            optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        loss_fn = nn.MSELoss()

        # Create DataLoader for batch training
        train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # -------------------------------
        # Training loop
        # -------------------------------
        for epoch in range(epochs):
            model.train()
            for xb, yb in train_loader:
                optimizer.zero_grad()
                loss = loss_fn(model(xb), yb)
                loss.backward()
                optimizer.step()

        # -------------------------------
        # Validation loss
        # -------------------------------
        model.eval()
        with torch.no_grad():
            val_loss = loss_fn(model(X_test), y_test).item()
            y_pred = model(X_test)
        
            y_true = y_test.cpu().numpy()
            y_pred = y_pred.cpu().numpy()

            mse = mean_squared_error(y_true, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)

        # -------------------------------
        # Log parameters and metrics
        # -------------------------------
        mlflow.log_params({
            "num_hidden_layers": num_hidden_layers,
            "neurons_per_layer": neurons_per_layer,
            "dropout_rate": dropout_rate,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "optimizer": optimizer_name,
            "weight_decay": weight_decay,
            "epochs": epochs
        })
        mlflow.log_metrics({
            "rmse": rmse,
            "mae": mae,
            "r2": r2
        })

        # Log model
        mlflow.pytorch.log_model(model, "model")

    return val_loss, rmse, mae, r2

In [14]:
mlflow.set_experiment("Optuna(r2,rmse)-PyTorch-Regression")

study = optuna.create_study(directions=["minimize","minimize","minimize", "maximize"])
study.optimize(objective, n_trials=5)

[I 2026-02-11 03:18:29,341] A new study created in memory with name: no-name-b1e0fe47-93ee-4f4a-aaff-3f95f9b9a653
2026/02/11 03:18:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 03:18:48 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run mysterious-crab-499 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2/runs/b2107f84331a4ac691c17321db0411d1
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2


[I 2026-02-11 03:18:53,436] Trial 0 finished with values: [0.6926822662353516, 0.8322753310203552, 0.6687746644020081, 0.31881529092788696] and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 112, 'dropout_rate': 0.1, 'learning_rate': 0.0003083134398353188, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 1.2576147203487068e-05, 'epochs': 50}.
2026/02/11 03:19:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 03:19:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run brawny-goat-400 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2/runs/d0a50fc1662d4ca39a9c8f5600ce0c3c
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2


[I 2026-02-11 03:19:32,453] Trial 1 finished with values: [0.9140339493751526, 0.956051230430603, 0.7647864818572998, 0.10113769769668579] and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 120, 'dropout_rate': 0.5, 'learning_rate': 1.3820143824602206e-05, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 0.0005246308089289776, 'epochs': 50}.
2026/02/11 03:19:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 03:20:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


...

[I 2026-02-11 03:20:06,238] Trial 2 finished with values: [0.2802785634994507, 0.5294134616851807, 0.4160975515842438, 0.7243736386299133] and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 72, 'dropout_rate': 0.1, 'learning_rate': 0.06328635729359712, 'batch_size': 16, 'optimizer': 'RMSprop', 'weight_decay': 0.0003422642829128717, 'epochs': 30}.
2026/02/11 03:20:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 03:20:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run caring-toad-348 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2/runs/95591143f654421897d0154e6e67ee0c
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2


[I 2026-02-11 03:20:54,620] Trial 3 finished with values: [0.2948436737060547, 0.5429950952529907, 0.42946022748947144, 0.7100503444671631] and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 8, 'dropout_rate': 0.5, 'learning_rate': 0.0003913006910040555, 'batch_size': 16, 'optimizer': 'RMSprop', 'weight_decay': 2.63971980540187e-05, 'epochs': 30}.
2026/02/11 03:20:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/11 03:21:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run clean-sheep-470 at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2/runs/1e9a3ee397dc47aca95805bc6adc6fab
🧪 View experiment at: https://dagshub.com/marufmullah50/Pytorch---DL_implementation.mlflow/#/experiments/2


[I 2026-02-11 03:21:40,377] Trial 4 finished with values: [0.14456802606582642, 0.38022100925445557, 0.30004870891571045, 0.857831597328186] and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 48, 'dropout_rate': 0.2, 'learning_rate': 0.002847906586326214, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 1.0362562619071438e-05, 'epochs': 10}.


In [ ]:
print(f"Number of Pareto-optimal trials: {len(study.best_trials)}")
# here multiple varialbles are maximized or minimized
for t in study.best_trials:
    print("Trial#", t.number)
    print("  Params:", t.params)
    print("  Values (RMSE, MAE, R2):", t.values)
    print()


Number of Pareto-optimal trials: 1
Trial# 4
  Params: {'num_hidden_layers': 3, 'neurons_per_layer': 48, 'dropout_rate': 0.2, 'learning_rate': 0.002847906586326214, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 1.0362562619071438e-05, 'epochs': 10}
  Values (RMSE, MAE, R2): [0.14456802606582642, 0.38022100925445557, 0.30004870891571045, 0.857831597328186]

